In [12]:
import numpy as np
import pandas as pd

In [13]:
#One-hot encoding the tendecy variable(3 categories). and dropping DR 
raw_data=pd.read_csv("../data/cleaned_data.csv")
tendecy_one_hot=pd.get_dummies(raw_data.loc[:,"Tendency"])
raw_data=pd.concat([raw_data,tendecy_one_hot],axis=1)
raw_data.drop(['Tendency','DR'],axis=1, inplace=True)
raw_data.columns

Index(['FileName', 'Date', 'SegFile', 'b', 'e', 'LBE', 'LB', 'AC', 'FM', 'UC',
       'ASTV', 'MSTV', 'ALTV', 'MLTV', 'DL', 'DS', 'DP', 'Width', 'Min', 'Max',
       'Nmax', 'Nzeros', 'Mode', 'Mean', 'Median', 'Variance', 'NSP',
       'left_assymetric', 'right_symmetric', 'symmetric'],
      dtype='object')

In [14]:
#Normalizing only the continuous variables
continuous_attributes=raw_data.iloc[:,6:26]
categorical_attributes=raw_data.iloc[:,26:]
continuous_attributes=(continuous_attributes-continuous_attributes.min())/(continuous_attributes.max()-continuous_attributes.min())
dataset=pd.concat([continuous_attributes,categorical_attributes.iloc[:,1:],categorical_attributes.iloc[:,0]],axis=1)
dataset.to_csv("../data/cleaned_normalized_data.csv",index=False)

In [15]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import style
style.use("ggplot")
from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import confusion_matrix, classification_report
import math

In [39]:
#dividing into train and test data
dataset_array=dataset.values
no_of_instances=len(dataset_array)
split=math.floor(0.6*no_of_instances)
data_x=dataset_array[:,:-1]
data_y=dataset_array[:,-1]
X_train=dataset_array[:split,:-1]
y_train=dataset_array[:split,-1]
X_test=dataset_array[split:,:-1]
y_test=dataset_array[split:,-1]

In [26]:
#Perform grid search to select the best values of C and gamma and the kernel function
from sklearn import svm, grid_search
from sklearn.model_selection import GridSearchCV
def svc_param_selection(X, y, nfolds):
    #Cs = [0.001, 0.01, 0.1, 1, 10]
    #gammas = [0.001, 0.01, 0.1, 1, 10, 100]
    Cs = [i for i in range(1,100)]
    gammas = [0.1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10 ]
    param_grid = {'C': Cs, 'gamma' : gammas}
    grid_search = GridSearchCV(svm.SVC(kernel='rbf'), param_grid, cv=nfolds)
    grid_search.fit(X, y)
    grid_search.best_params_
    return grid_search.best_params_
svc_param_selection(data_x,data_y,5)

{'C': 33, 'gamma': 0.1}

In [40]:
#Define the SVC classifier using sklearn package
classifier = svm.SVC(kernel='rbf', C = 4.0,gamma=2.0, verbose=True)

#Training the model
classifier.fit(X_train,y_train)

#testing the model
y_pred=classifier.predict(X_test)

#getting the classification metrics
confusion_mtx = confusion_matrix(y_test, y_pred) 
print("\nConfusion Matrix=\n",confusion_mtx)
target_names = ['normal', 'suspect', 'pathological']
print(classification_report(y_test, y_pred, target_names=target_names))
print("accuracy= ",(confusion_mtx.diagonal().sum()/confusion_mtx.sum())*100)



[LibSVM]
Confusion Matrix=
 [[639  68   8]
 [ 21  21   0]
 [ 21   3  70]]
              precision    recall  f1-score   support

      normal       0.94      0.89      0.92       715
     suspect       0.23      0.50      0.31        42
pathological       0.90      0.74      0.81        94

 avg / total       0.90      0.86      0.87       851

accuracy=  85.7814336075
